In [1]:
import json, re

from googleTranslate import translate

In [2]:
def transEgs(Eg):
    trans = translate(Eg)
    if trans != "Error":
        return trans["sentences"][0]["trans"]
    else:
        print(trans)
        return 0


In [3]:

with open("wikiData.json") as f:
    wikiData = json.loads(f.read())

with open("wiki_data.json") as f:
    wiki_data = json.loads(f.read())

word_list = list(set(list(wiki_data.keys())).difference(list(wikiData.keys())))


In [ ]:
count = len(word_list) 
for word in word_list:
    print(count, word)
    count -= 1
    meanings = []
    try:
        for meaning in wiki_data[word]["meanings"]:
            meaning["ejemplos"] = list(map(transEgs, meaning["examples"]))
            meanings.append(meaning)
        wiki_data[word]["meanings"] = meanings
    except:
        continue

In [6]:
with open("wiki_data_trans.json", 'w') as f:
    json.dump(wiki_data, f, ensure_ascii=False)

In [4]:
import json, re
with open("googleDict.json", 'r') as googleDict:
    googleDict = json.loads(googleDict.read())
with open("wikiData_trans.json", 'r') as wikiData:
    wikiData = json.loads(wikiData.read())


    def delSomeEgs(meaning, translations):
        ej_list = meaning["ejemplos"]
        eg_list = meaning["examples"]
        newEg_list = []
        newEj_list = []
        trans_terms = []
        index = 0
        count = 0
        used_twice = []
        for ejemplo in ej_list:
            if count > 4:
                break
            for trans in translations:
                if trans in used_twice:
                    continue
                if re.search(rf"\b{trans}\b|\b{trans.capitalize()}\b",ejemplo):
                    newEg_list.append(eg_list[index])
                    newEj_list.append(ejemplo)
                    if trans in trans_terms:
                        used_twice.append(trans)
                    trans_terms.append(trans) 
                    count += 1
                    break
            index += 1
        meaning["examples"] = newEg_list
        meaning["ejemplos"] = newEj_list
        meaning["trans_terms"] = trans_terms
        return meaning

    
    classes = {
    'article': 'artículo',
    'preposition': 'preposición',
    'adverb': 'adverbio',
    'adjective': 'adjetivo',
    'noun': 'sustantivo',
    'conjunction': 'conjunción',
    'determiner': 'adjetivo',
    'pronoun': 'pronombre',
    'verb': 'verbo',
    'proper noun': 'sustantivo',
    'numeral': 'sustantivo'
    }

    for word in wikiData:
        word_meanings = []
        for meaning in wikiData[word]["meanings"]:
            pos = meaning["class"]
            if pos == "interjection" or pos == "symbol":
                continue
            pos = classes[pos]
            if "dict" in googleDict[word]:
                for entry in googleDict[word]["dict"]:
                    if entry["pos"] == pos:
                        word_meanings.append(delSomeEgs(meaning, entry["terms"]))                     
        wikiData[word]["meanings"] = word_meanings


with open("ownDict.json", "w") as f:
    json.dump(wikiData, f, ensure_ascii = False)

In [ ]:

# with open("wikiData.json", "r") as f:
#     wikiData = json.loads(f.read())
#     classes = []
#     for word in wikiData:
#         for meaning in wikiData[word]["meanings"]:
#             wordClass = meaning["class"]
#             if wordClass not in classes:
#                 classes.append(wordClass)

# print(classes)

In [ ]:

# with open("googleDict.json", "r") as f:
#     Data = json.loads(f.read())
#     classes = []
#     for word in Data:
#         if "dict" in Data[word]:
#             for meaning in Data[word]["dict"]:
#                 wordClass = meaning["pos"]
#                 if wordClass not in classes:
#                     classes.append(wordClass)

# print(classes)